In [3]:

# download packages
import io
import pandas as pd
from PIL import Image
import numpy as np


In [4]:
# load dataset, make sure notebook and file are in the same folder
# data = pd.read_csv("styles.csv", on_bad_lines = 'skip')

data = pd.read_csv("styles.csv", on_bad_lines = 'skip')
# rename columns, remove year
data.rename(columns = {"masterCategory" : "category", "articleType" : "article", "baseColour" : "color"}, inplace = True)
data.drop(columns = ["year", "usage", "gender", "color", "productDisplayName"], inplace = True)

In [5]:
# keep apparel and footwear items, seperate df
d_clean = data[(data["category"] == "Apparel") | (data["category"] == "Footwear")]
# remove innerwear
d_clean = d_clean[d_clean["subCategory"] != "Innerwear"]


In [6]:

# changes apparel into top/bottom/one
d_clean.loc[d_clean["subCategory"] == "Topwear", "category"] = "Top"
d_clean.loc[d_clean["subCategory"] == "Bottomwear", "category"] = "Bottom"
d_clean.loc[d_clean["category"] == "Apparel", "category"] = "One"


In [7]:
# condense article types
# need to condense further
d_clean.loc[d_clean["category"] == "Footwear", "article"] = "Shoes"


In [8]:

# changes the season into numbers
d_num = d_clean
d_num.loc[d_num["season"].isna(), "season"] = "Summer"
k_sea = d_num["season"].unique()
k_sea = {key: i for i, key in enumerate(k_sea)}
d_num["season"] = d_num["season"].map(k_sea)

k_sea

{'Fall': 0, 'Summer': 1, 'Winter': 2, 'Spring': 3}

In [11]:
def resize_grayscale_binary(image_id, new_width = 50, new_height = 50):
    """
    Opens an image from the given file path and resizes it.

    Parameters:
        image_id (str): Path to the image file.
        new_width (int): Desired width in pixels.
        new_height (int): Desired height in pixels.

    Returns:
        Image: The resized PIL Image object.
    """
    try:
        # Open the image
        img = Image.open(rf"C:\Users\doncv\Desktop\AISC\images\{image_id}.jpg")
        # Resize the image
        binary = img.resize((new_width, new_height)).convert("L").point(lambda p: 255 if p > 128 else 0)

        return np.array(binary).flatten()

    except FileNotFoundError:
        print("Error: The specified image file was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [10]:
article_mapping = {
    # 👕 Tops
    "Tshirts": "Tee",
    "Tops": "Tee",
    "Lounge Tshirts": "Tee",

    "Shirts": "Shirt",
    "Kurtas": "Shirt",
    "Kurtis": "Shirt",
    "Tunics": "Shirt",

    # 👖 Bottoms
    "Trousers": "Trouser",
    "Track Pants": "Sweatpant",
    "Lounge Pants": "Sweatpant",
    "Tracksuits": "Sweatpant",
    "Jeans": "Jean",
    "Jeggings": "Jean",
    "Leggings": "Legging",
    "Churidar": "Legging",
    "Tights": "Legging",
    "Patiala": "Trouser",
    "Salwar": "Trouser",

    # 👗 Dress & Skirt
    "Dresses": "Dress",
    "Sarees": "Dress",
    "Kurta Sets": "Dress",
    "Lehenga Choli": "Dress",
    "Nightdress": "Dress",
    "Jumpsuit": "Dress",
    "Rompers": "Dress",

    "Skirts": "Skirt",
    "Shorts": "Short",
    "Lounge Shorts": "Short",

    # 🧥 Outerwear
    "Jackets": "Jacket",
    "Waistcoat": "Jacket",
    "Nehru Jackets": "Jacket",
    "Shrug": "Jacket",

    "Blazers": "Blazer",
    "Rain Jacket": "Rain Jacket",

    # 👜 Accessories / Others
    "Handbags": "Bag",
    "Laptop Bag": "Bag",
    "Caps": "Headwear",
    "Hats": "Headwear",

    # 👟 Shoes
    "Shoes": "Shoe",
    "Casual Shoes": "Shoe",
    "Formal Shoes": "Shoe",
    "Sports Shoes": "Shoe",
    "Heels": "Shoe",
    "Flats": "Shoe",
    "Sandals": "Shoe",
    "Flip Flops": "Shoe",
    "Booties": "Shoe"
}

d_num["article"] = d_num["article"].map(article_mapping).fillna("Other")
# changes the articles into numbers
k_art = d_num["article"].unique()
k_art = {key: i for i, key in enumerate(k_art)}
d_num["article"] = d_num["article"].map(k_art)

k_art

{'Shirt': 0,
 'Jean': 1,
 'Sweatpant': 2,
 'Tee': 3,
 'Shoe': 4,
 'Other': 5,
 'Jacket': 6,
 'Short': 7,
 'Dress': 8,
 'Rain Jacket': 9,
 'Skirt': 10,
 'Blazer': 11,
 'Trouser': 12,
 'Legging': 13}

In [12]:
binary_img = {}
ids = d_num["id"].unique()
for id in ids:
    binary_img[id] = resize_grayscale_binary(id)

Error: The specified image file was not found.
Error: The specified image file was not found.
Error: The specified image file was not found.
Error: The specified image file was not found.
Error: The specified image file was not found.


In [ ]:
d_num.to_csv("filename.csv", index=False)

In [19]:
binary_img = {k: v for k, v in binary_img.items() if v is not None}
df = pd.DataFrame.from_dict(binary_img, orient="index")


In [20]:
d_num.merge(df, left_on = "id", right_index = True, how = "right")

,id,category,subCategory,article,season,0,1,2,3,4,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,15970,Top,Topwear,0,0,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,39386,Bottom,Bottomwear,1,1,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,21379,Bottom,Bottomwear,2,0,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,53759,Top,Topwear,3,1,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
5,1855,Top,Topwear,3,1,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44417,12544,Top,Topwear,3,0,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
44418,42234,Top,Topwear,3,1,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
44419,17036,Footwear,Shoes,4,1,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
44420,6461,Footwear,Flip Flops,4,1,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [24]:
d_num.drop(columns = ["subCategory", "category"], inplace = True)

In [25]:
d_num.to_csv("clean.csv", index=False)